# Wait


We scrape data from [CA DMV](https://www.dmv.ca.gov/portal/field-office/woodland/) to answer whether the number of services, the average wait time, the hours a facility is open, etc. vary by income.
 
## Data
 
There are 175 DMV field offices in CA. There are [178 DMV offices](data/yogov_dmv_list.txt) listed on https://yogov.org/dmv/california/california-dmv-locations/. However a couple have closed since yogov compiled their list. 

For each DMV field office we collect...
- basic data: "name", "street", "locality", "region", and "zip".
- wait time (minutes) by hour: wait time for the Monday 2pm hour is stored in the column "M14".
- eight potential services offered: "title transfers", "licensing services", "replace lost/stolen/damaged", "plates permits & placards", "testing", "records", "registration", "request for miscellaneous original documents."


## Analysis

We estimate the average wait time, averaging over the average for all hours (days). We also estimate the 25th and 75th percentile of wait times for each location.

Next, to assess whether the staffing levels are potentially suboptimal, we estimate average by hour and by weekday, again averaging across locations.

Next, we download town level data on sociodemographics and plot a loess between median income, % African Americans, etc. and the number of services offered, total open time, average wait time, etc. 

## Authors 

Noah Finberg and Gaurav Sood


## Load Data

In [ ]:
import pandas as pd

In [ ]:
dmv_df = pd.read_csv("data/dmv_data_output_12_14_2020.csv", index_col=0)

In [ ]:
dmv_df.head()

## Descriptive Statistics

#### Define Helper Cols

In [69]:
# define columns
# 168 wait time columns
wait_time_cols = ['SU0', 'SU1', 'SU2', 'SU3', 'SU4', 'SU5', 'SU6', 'SU7', 'SU8', 'SU9', 'SU10', 'SU11', 'SU12',
'SU13', 'SU14', 'SU15', 'SU16', 'SU17', 'SU18', 'SU19', 'SU20', 'SU21', 'SU22', 'SU23', 
'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 
'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 
'TU0', 'TU1', 'TU2', 'TU3', 'TU4', 'TU5', 'TU6', 'TU7', 'TU8', 'TU9', 'TU10', 
'TU11', 'TU12', 'TU13', 'TU14', 'TU15', 'TU16', 'TU17', 'TU18', 'TU19', 'TU20', 'TU21', 'TU22', 'TU23', 
'W0', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 
'W14', 'W15', 'W16', 'W17', 'W18', 'W19', 'W20', 'W21', 'W22', 'W23', 
'TH0', 'TH1', 'TH2', 'TH3', 'TH4', 'TH5', 'TH6', 'TH7', 'TH8', 'TH9', 'TH10', 'TH11', 'TH12', 'TH13',
'TH14', 'TH15', 'TH16', 'TH17', 'TH18', 'TH19', 'TH20', 'TH21', 'TH22', 'TH23',
'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 
'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 
'SA0', 'SA1', 'SA2', 'SA3', 'SA4', 'SA5', 'SA6', 'SA7', 'SA8', 'SA9', 'SA10', 'SA11', 'SA12',
'SA13', 'SA14', 'SA15', 'SA16', 'SA17', 'SA18', 'SA19', 'SA20', 'SA21', 'SA22', 'SA23']

# 120 weekday cols
weekday_cols = ['M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 
'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23', 
'TU0', 'TU1', 'TU2', 'TU3', 'TU4', 'TU5', 'TU6', 'TU7', 'TU8', 'TU9', 'TU10', 
'TU11', 'TU12', 'TU13', 'TU14', 'TU15', 'TU16', 'TU17', 'TU18', 'TU19', 'TU20', 'TU21', 'TU22', 'TU23', 
'W0', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 
'W14', 'W15', 'W16', 'W17', 'W18', 'W19', 'W20', 'W21', 'W22', 'W23', 
'TH0', 'TH1', 'TH2', 'TH3', 'TH4', 'TH5', 'TH6', 'TH7', 'TH8', 'TH9', 'TH10', 'TH11', 'TH12', 'TH13',
'TH14', 'TH15', 'TH16', 'TH17', 'TH18', 'TH19', 'TH20', 'TH21', 'TH22', 'TH23',
'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 
'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', ]

sun_cols = ['SU0', 'SU1', 'SU2', 'SU3', 'SU4', 'SU5', 'SU6', 'SU7', 'SU8', 'SU9', 'SU10', 'SU11', 'SU12',
'SU13', 'SU14', 'SU15', 'SU16', 'SU17', 'SU18', 'SU19', 'SU20', 'SU21', 'SU22', 'SU23']
mon_cols = ['M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 
'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20', 'M21', 'M22', 'M23',]
tues_cols = ['TU0', 'TU1', 'TU2', 'TU3', 'TU4', 'TU5', 'TU6', 'TU7', 'TU8', 'TU9', 'TU10', 
'TU11', 'TU12', 'TU13', 'TU14', 'TU15', 'TU16', 'TU17', 'TU18', 'TU19', 'TU20', 'TU21', 'TU22', 'TU23',]
wed_cols = ['W0', 'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 
'W14', 'W15', 'W16', 'W17', 'W18', 'W19', 'W20', 'W21', 'W22', 'W23', ]
thurs_cols = ['TH0', 'TH1', 'TH2', 'TH3', 'TH4', 'TH5', 'TH6', 'TH7', 'TH8', 'TH9', 'TH10', 'TH11', 'TH12', 'TH13',
'TH14', 'TH15', 'TH16', 'TH17', 'TH18', 'TH19', 'TH20', 'TH21', 'TH22', 'TH23',]
fri_cols = ['F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 
'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23',]
sat_cols = ['SA0', 'SA1', 'SA2', 'SA3', 'SA4', 'SA5', 'SA6', 'SA7', 'SA8', 'SA9', 'SA10', 'SA11', 'SA12',
'SA13', 'SA14', 'SA15', 'SA16', 'SA17', 'SA18', 'SA19', 'SA20', 'SA21', 'SA22', 'SA23']

# hour cols dictionary hour_cols[hour] = [cols]
hour_cols = {}
for i in range(24):
    hour_cols[i] = ['SU' + str(i),'M'+ str(i),'TU'+ str(i), 'W'+ str(i), 'TH'+ str(i), 'F'+ str(i), "SA"+ str(i)]
    
# weekday_hour cols dict
weekday_hour_cols_dict = {}
for i in range(24):
    weekday_hour_cols_dict[i] = ['M'+ str(i),'TU'+ str(i), 'W'+ str(i), 'TH'+ str(i), 'F'+ str(i)]

### Average wait time

The average wait time, averaging over the average for all hours (days) and the 25th and 75th percentile of wait times for each location.

#### Overall Descriptive Stats by Location

In [63]:
# Get Descriptive Stats for Each Location
# Note: could get min and max as well, but thought that might be overkill in terms of no. of columns

wait_time_df = dmv_df[wait_time_cols]

# overall descriptive statistics per location (25th, 75th, and std)
daily_descriptive_stats = wait_time_df.apply(pd.DataFrame.describe, axis=1)
weekday_descriptive_stats = wait_time_df[weekday_cols].apply(pd.DataFrame.describe, axis=1)


# what is the rough distribution of wait_times by day by location? 
# daily_wait_std, daily_wait_25, daily_wait_50 daily_wait_75
wait_time_df["daily_wait_std"] = daily_descriptive_stats["std"]
wait_time_df["daily_wait_25"] = daily_descriptive_stats["25%"]
wait_time_df["daily_wait_50"] = daily_descriptive_stats["50%"]
wait_time_df["daily_wait_75"] = daily_descriptive_stats["75%"]

# what is the rough distribution of wait_times by day for only weekdays by location? 
# weekday_wait_std, weekday_wait_25, weekday_wait_50, weekday_wait_75
wait_time_df["weekday_wait_std"] = weekday_descriptive_stats["std"]
wait_time_df["weekday_wait_25"] = weekday_descriptive_stats["25%"]
wait_time_df["weekday_wait_50"] = weekday_descriptive_stats["50%"]
wait_time_df["weekday_wait_75"] = weekday_descriptive_stats["75%"]

wait_time_df.head()


/Users/noahfinberg/Desktop/GauravCollab/env/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/noahfinberg/Desktop/GauravCollab/env/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/noahfinberg/Desktop/GauravCollab/env/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,SU0,SU1,SU2,SU3,SU4,SU5,SU6,SU7,SU8,SU9,...,SA22,SA23,daily_wait_std,daily_wait_25,daily_wait_50,daily_wait_75,weekday_wait_std,weekday_wait_25,weekday_wait_50,weekday_wait_75
name,,,,,,,,,,,,,,,,,,,,,
Alturas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.774698,0.0,0.0,1.00,0.789049,0.00,1.0,1.00
Arleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14.939255,0.0,14.5,31.00,14.205305,7.00,20.0,31.00
Arvin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.776450,0.0,10.0,24.00,12.510098,4.00,13.5,25.25
Auburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.355075,0.0,11.5,20.00,12.106589,1.50,15.5,21.00
Bakersfield Southwest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,15.134349,0.0,15.0,29.25,14.658678,3.75,17.5,30.25


#### Average Wait Times By Day by Location

In [64]:
# how long is the average wait on a given day by location?
wait_time_df["sun_avg"] = wait_time_df[sun_cols].mean(axis=1)
wait_time_df["mon_avg"] = wait_time_df[mon_cols].mean(axis=1)
wait_time_df["tues_avg"] = wait_time_df[tues_cols].mean(axis=1)
wait_time_df["wed_avg"] = wait_time_df[wed_cols].mean(axis=1)
wait_time_df["thurs_avg"] = wait_time_df[thurs_cols].mean(axis=1)
wait_time_df["fri_avg"] = wait_time_df[fri_cols].mean(axis=1)
wait_time_df["sat_avg"] = wait_time_df[sat_cols].mean(axis=1)

/Users/noahfinberg/Desktop/GauravCollab/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/noahfinberg/Desktop/GauravCollab/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/noahfinberg/Desktop/GauravCollab/env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be

#### Average Wait Times by Hour by Location

In [73]:
# how long is the average wait on a given hour by location? 
# new column = wait_time_df[0_hour_avg_wait] = avg of waits across 0th hour

# NOTE: Just focused on weekdays for now given there is little to no wait on weekends. Can easily add those day if you'd like.

for i in range(24):
    weekday_hour_col_name = str(i) +"_weekday_hour_avg_wait"
    weekday_hour_cols = weekday_hour_cols_dict[i]
    print(weekday_hour_col_name)
    print(weekday_hour_cols)
    wait_time_df[weekday_hour_col_name] = wait_time_df[weekday_hour_cols].mean(axis=1)
    
wait_time_df.head()

0_weekday_hour_avg_wait
['M0', 'TU0', 'W0', 'TH0', 'F0']
1_weekday_hour_avg_wait
['M1', 'TU1', 'W1', 'TH1', 'F1']
2_weekday_hour_avg_wait
['M2', 'TU2', 'W2', 'TH2', 'F2']
3_weekday_hour_avg_wait
['M3', 'TU3', 'W3', 'TH3', 'F3']
4_weekday_hour_avg_wait
['M4', 'TU4', 'W4', 'TH4', 'F4']
5_weekday_hour_avg_wait
['M5', 'TU5', 'W5', 'TH5', 'F5']
6_weekday_hour_avg_wait
['M6', 'TU6', 'W6', 'TH6', 'F6']
7_weekday_hour_avg_wait
['M7', 'TU7', 'W7', 'TH7', 'F7']
8_weekday_hour_avg_wait
['M8', 'TU8', 'W8', 'TH8', 'F8']
9_weekday_hour_avg_wait
['M9', 'TU9', 'W9', 'TH9', 'F9']
10_weekday_hour_avg_wait
['M10', 'TU10', 'W10', 'TH10', 'F10']
11_weekday_hour_avg_wait
['M11', 'TU11', 'W11', 'TH11', 'F11']
12_weekday_hour_avg_wait
['M12', 'TU12', 'W12', 'TH12', 'F12']
13_weekday_hour_avg_wait
['M13', 'TU13', 'W13', 'TH13', 'F13']
14_weekday_hour_avg_wait
['M14', 'TU14', 'W14', 'TH14', 'F14']
15_weekday_hour_avg_wait
['M15', 'TU15', 'W15', 'TH15', 'F15']
16_weekday_hour_avg_wait
['M16', 'TU16', 'W16', 'TH1

,SU0,SU1,SU2,SU3,SU4,SU5,SU6,SU7,SU8,SU9,...,14_weekday_hour_avg_wait,15_weekday_hour_avg_wait,16_weekday_hour_avg_wait,17_weekday_hour_avg_wait,18_weekday_hour_avg_wait,19_weekday_hour_avg_wait,20_weekday_hour_avg_wait,21_weekday_hour_avg_wait,22_weekday_hour_avg_wait,23_weekday_hour_avg_wait
name,,,,,,,,,,,,,,,,,,,,,
Alturas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.2,1.0,0.6,0.0,0.0,NaN,NaN,NaN,NaN,NaN
Arleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28.0,35.2,17.6,3.4,0.0,NaN,NaN,NaN,NaN,NaN
Arvin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.2,20.8,9.0,0.4,0.0,NaN,NaN,NaN,NaN,NaN
Auburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.0,22.0,12.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN
Bakersfield Southwest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.8,28.8,12.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN


## Write Descriptive Stats to CSV

In [75]:
wait_time_df.to_csv("data/dmv_wait_descriptive_stats.csv")